In [1]:
import sys
import os

In [2]:
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)

In [3]:
import json

In [4]:
with open("../dataset/PIZZA_train_sampled.json", 'r') as infile, open("../dataset/PIZZA_train_2.json", 'w') as outfile:

    line_count = 0
    processed_count = 0
    batch = []

    for line in infile:

        instance = json.loads(line)
        top_field = instance.get(f"train.TOP", "")

        labels = []
        orders = []

        stack = []  # To track all special tokens

        i = 0
        flag = False
        topping_begin = False
        while i < len(top_field):
            return_topping = False
            char = top_field[i]
            if char == '(':
                # Found the start of a new group
                i += 1
                group_start = i
                while i < len(top_field) and top_field[i] != ' ':
                    i += 1
                entity = top_field[group_start:i]
                stack.append(entity)
                if entity in {"PIZZAORDER", "DRINKORDER"}:
                    flag = True
                    orders.append([])
                    labels.append([])
                elif entity == "TOPPING":
                    topping_begin = True
            elif char == ')':
                # End of the current entity
                if stack and stack[-1] in {"PIZZAORDER", "DRINKORDER"}:
                    flag = False
                if stack:
                    stack.pop()
                i += 1
            elif char == ' ':
                # Skip spaces
                i += 1
            else:
                # Process a token
                group_start = i
                while i < len(top_field) and top_field[i] != ' ':
                    i += 1
                word = top_field[group_start:i]

                if flag:
                    orders[-1].append(word)
                    if stack and topping_begin:
                        stack[-1] = "TOPPING_BEGIN"
                        topping_begin = False
                        return_topping = True
                    elif stack and stack[-1] == "TOPPING":
                        stack[-1] = "TOPPING_INTERMEDIATE"
                        return_topping = True
                    label = '-'.join(stack[2:])
                    if return_topping: stack[-1] = "TOPPING"
                    if label == "" or label == "NOT": label = "OTHER"
                    labels[-1].append(label)
        
        for i in range(len(orders)):
            item = {
                "train.SRC": ' '.join(orders[i]),
                "train.LABELS": labels[i]
            }
            outfile.write(json.dumps(item) + '\n')


In [5]:
with open("../dataset/PIZZA_train_2.json", 'r') as infile:
    unique_labels = set()

    for line in infile:
        instance = json.loads(line)
        labels_field = instance.get(f"train.LABELS", "")
        for label in labels_field: unique_labels.add(label)


print(unique_labels)

    

{'OTHER', 'VOLUME', 'COMPLEX_TOPPING-QUANTITY', 'CONTAINERTYPE', 'COMPLEX_TOPPING-TOPPING_BEGIN', 'DRINKTYPE', 'NOT-STYLE', 'COMPLEX_TOPPING-TOPPING_INTERMEDIATE', 'NOT-COMPLEX_TOPPING-QUANTITY', 'TOPPING_BEGIN', 'NOT-COMPLEX_TOPPING-TOPPING_BEGIN', 'STYLE', 'SIZE', 'NUMBER', 'NOT-TOPPING_INTERMEDIATE', 'NOT-TOPPING_BEGIN', 'TOPPING_INTERMEDIATE'}


In [6]:
with open("../dataset/PIZZA_test.json", 'r') as infile, open("../dataset/PIZZA_test_2.json", 'w') as outfile:

    line_count = 0
    processed_count = 0
    batch = []

    for line in infile:

        instance = json.loads(line)
        top_field = instance.get(f"test.TOP", "")

        labels = []
        orders = []

        stack = []  # To track all special tokens

        i = 0
        flag = False
        topping_begin = False
        while i < len(top_field):
            return_topping = False
            char = top_field[i]
            if char == '(':
                # Found the start of a new group
                i += 1
                group_start = i
                while i < len(top_field) and top_field[i] != ' ':
                    i += 1
                entity = top_field[group_start:i]
                stack.append(entity)
                if entity in {"PIZZAORDER", "DRINKORDER"}:
                    flag = True
                    orders.append([])
                    labels.append([])
                elif entity == "TOPPING":
                    topping_begin = True
            elif char == ')':
                # End of the current entity
                if stack and stack[-1] in {"PIZZAORDER", "DRINKORDER"}:
                    flag = False
                if stack:
                    stack.pop()
                i += 1
            elif char == ' ':
                # Skip spaces
                i += 1
            else:
                # Process a token
                group_start = i
                while i < len(top_field) and top_field[i] != ' ':
                    i += 1
                word = top_field[group_start:i]

                if flag:
                    orders[-1].append(word)
                    if stack and topping_begin:
                        stack[-1] = "TOPPING_BEGIN"
                        topping_begin = False
                        return_topping = True
                    elif stack and stack[-1] == "TOPPING":
                        stack[-1] = "TOPPING_INTERMEDIATE"
                        return_topping = True
                    label = '-'.join(stack[2:])
                    if return_topping: stack[-1] = "TOPPING"
                    if label == "" or label == "NOT": label = "OTHER"
                    labels[-1].append(label)
        
        for i in range(len(orders)):
            item = {
                "test.SRC": ' '.join(orders[i]),
                "test.LABELS": labels[i]
            }
            outfile.write(json.dumps(item) + '\n')


In [7]:
with open("../dataset/PIZZA_test_2.json", 'r') as infile:
    unique_labels_2 = set()

    for line in infile:
        instance = json.loads(line)
        labels_field = instance.get(f"test.LABELS", "")
        for label in labels_field: unique_labels_2.add(label)


print(unique_labels_2)

    

{'OTHER', 'VOLUME', 'COMPLEX_TOPPING-QUANTITY', 'CONTAINERTYPE', 'COMPLEX_TOPPING-TOPPING_BEGIN', 'DRINKTYPE', 'NOT-STYLE', 'NOT-COMPLEX_TOPPING-QUANTITY', 'NOT-COMPLEX_TOPPING-TOPPING_BEGIN', 'TOPPING_BEGIN', 'COMPLEX_TOPPING-TOPPING_INTERMEDIATE', 'STYLE', 'SIZE', 'NUMBER', 'NOT-TOPPING_INTERMEDIATE', 'NOT-TOPPING_BEGIN', 'TOPPING_INTERMEDIATE'}


In [8]:
unique_labels

{'COMPLEX_TOPPING-QUANTITY',
 'COMPLEX_TOPPING-TOPPING_BEGIN',
 'COMPLEX_TOPPING-TOPPING_INTERMEDIATE',
 'CONTAINERTYPE',
 'DRINKTYPE',
 'NOT-COMPLEX_TOPPING-QUANTITY',
 'NOT-COMPLEX_TOPPING-TOPPING_BEGIN',
 'NOT-STYLE',
 'NOT-TOPPING_BEGIN',
 'NOT-TOPPING_INTERMEDIATE',
 'NUMBER',
 'OTHER',
 'SIZE',
 'STYLE',
 'TOPPING_BEGIN',
 'TOPPING_INTERMEDIATE',
 'VOLUME'}

In [9]:
unique_labels_2

{'COMPLEX_TOPPING-QUANTITY',
 'COMPLEX_TOPPING-TOPPING_BEGIN',
 'COMPLEX_TOPPING-TOPPING_INTERMEDIATE',
 'CONTAINERTYPE',
 'DRINKTYPE',
 'NOT-COMPLEX_TOPPING-QUANTITY',
 'NOT-COMPLEX_TOPPING-TOPPING_BEGIN',
 'NOT-STYLE',
 'NOT-TOPPING_BEGIN',
 'NOT-TOPPING_INTERMEDIATE',
 'NUMBER',
 'OTHER',
 'SIZE',
 'STYLE',
 'TOPPING_BEGIN',
 'TOPPING_INTERMEDIATE',
 'VOLUME'}

In [10]:
len(unique_labels)

17

In [11]:
len(unique_labels_2)

17